<a href="https://colab.research.google.com/github/Vihith123/Gemma-PreTrained/blob/main/Gemma3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

1. Importing the Datasets

In [2]:
pip install -U datasets

In [3]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

2. Tokenization of the Dataset


In [4]:
!pip install tiktoken
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm
from pathlib import Path
import pandas as pd
enc = tiktoken.get_encoding("gpt2")
def process(example):
  ids = enc.encode_ordinary(example['text'])
  out = {'ids': ids, 'len': len(ids)}
  return out

if not os.path.exists("train.bin"):
    tokenized = ds.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=8,
        )

for split, dset in tokenized.items(): # Iterating through the Dataset Splits
        arr_len = np.sum(dset['len'], dtype=np.uint64) # Counting the number of tokens in the dset after spliting
        filename = f'{split}.bin' # Creating two bins i.e bag of words train.bin and val.bin
        dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16) we have max tokens as 50257 as int16 can 65k so it better
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,)) # here mode w+ is to create a bin if not created and write the token ids into thr bin if the bin is already created
        # IT deletes and creates a new bin
        total_batches = 1024 # Divided into the batches

        idx = 0 # Assinging the idx to batches
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'): # Iterating along the batches in the specified bins
            # Batch together samples for faster write
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy') # It creates in any contigous 1D array manner
            arr_batch = np.concatenate(batch['ids']) #After Shards it concats the ids
            # Write into mmap
            arr[idx : idx + len(arr_batch)] = arr_batch # It is Copying the token ids into the arr.memmap asn arr was empty and arr_batc contained the token ids
            idx += len(arr_batch) # It is incrementing each one
        arr.flush()

tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

writing train.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

writing validation.bin:   0%|          | 0/1024 [00:00<?, ?it/s]